# Creación de un Entorno de Python

En este documento, vamos a explicar cómo configurar un nuevo entorno de Python utilizando tanto `venv` como `conda`.

## Usando venv

`venv` es una herramienta que viene preinstalada con Python (versión 3.3 o superior) y permite crear entornos virtuales de Python aislados.

### Pasos

1. **Crear un entorno virtual:** Abre la terminal y ejecuta el siguiente comando en el directorio donde desees crear el entorno virtual.

   ```bash
   python3 -m venv nombre_del_entorno


2. **Activar el entorno virtual:** Utiliza el siguiente comando para activar el entorno.


  Linux/Mac:

```bash
    source nombre_del_entorno/bin/activate
```


   Windows:

```bash
    .\nombre_del_entorno\Scripts\activate
```


3. **Instalar dependencias desde requirements.txt con pip:** Una vez activado el entorno, puedes instalar todas las dependencias necesarias desde un archivo `requirements.txt` utilizando:

   ```bash
   pip install -r requirements.txt

4. **Desactivar el entorno virtual:** Para salir del entorno virtual, simplemente ejecuta:

```bash
   deactivate

## Usando Conda
conda es un sistema de gestión de paquetes y de entorno que puede instalar paquetes de diferentes lenguajes.

### Pasos
1. **Instalar Conda:** Puedes descargar e instalar Conda desde [este enlace](https://www.anaconda.com/download).

2. **Crear un entorno Conda:** Para crear un nuevo entorno con Conda, abre la terminal y ejecuta:

```bash
    conda create --name nombre_del_entorno python=3.9 ipykernel
```

Para esta biblioteca, vamos a usar python 3.9 y para interactuar con Jupyter Notebooks vamos a usar "ipykernel" o "jupyter"

3. **Activar el entorno Conda:** Utiliza el siguiente comando para activar el entorno.

```bash
    conda activate nombre_del_entorno
```
4. **Instalar dependencias desde requirements.txt con pip:** Al igual que con venv, puedes instalar las dependencias necesarias desde un archivo requirements.txt en el entorno Conda activado utilizando:
```bash
    pip install -r requirements.txt
```

5. Desactivar el entorno Conda: Para salir del entorno Conda, simplemente ejecuta:

```bash
    conda deactivate
```

Importamos las librerías necesarias

In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer, InputExample
import numpy as np
import faiss

c:\Users\op211673\Anaconda3\envs\llm-prd\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Newscatcher Dataset

El conjunto de datos "Newscatcher" se refiere a una colección de noticias recopiladas de varias fuentes y medios de comunicación. Puede incluir información como el título de la noticia, la descripción, el contenido completo del artículo, la fecha de publicación, el autor y la URL de origen.

La utilidad de este conjunto de datos radica en su aplicación en diversos campos como el análisis de sentimientos, la clasificación de noticias, el seguimiento de tendencias y temas emergentes, entre otros. Es especialmente útil en el procesamiento del lenguaje natural (NLP) y en el análisis de datos, donde se pueden aplicar técnicas de aprendizaje automático para extraer insights y patrones significativos.

Este conjunto de datos puede encontrarse en diversos formatos y se puede personalizar para incluir noticias de temas específicos, regiones geográficas o periodos de tiempo.

[Newscatcher GitHub](https://github.com/kotartemiy/newscatcher) proporciona bibliotecas para trabajar con este tipo de datos, facilitando la recopilación y el procesamiento de noticias de miles de fuentes confiables.


In [2]:
pdf = pd.read_csv(f"labelled_newscatcher_coloured.csv", index_col=0)

In [3]:

pdf["id"] = pdf.index
display(pdf)


,topic,link,domain,published_date,title,lang,id
0,SCIENCE,https://www.eurekalert.org/pub_releases/2020-0...,eurekalert.org,2020-08-06 13:59:45,A closer look at water-splitting's solar fuel ...,en,0
1,SCIENCE,https://www.pulse.ng/news/world/an-irresistibl...,pulse.ng,2020-08-12 15:14:19,"An irresistible scent makes locusts swarm, stu...",en,1
2,SCIENCE,https://www.express.co.uk/news/science/1322607...,express.co.uk,2020-08-13 21:01:00,Artificial intelligence warning: AI will know ...,en,2
3,SCIENCE,https://www.ndtv.com/world-news/glaciers-could...,ndtv.com,2020-08-03 22:18:26,Glaciers Could Have Sculpted Mars Valleys: Study,en,3
4,SCIENCE,https://www.thesun.ie/tech/5742187/perseid-met...,thesun.ie,2020-08-12 19:54:36,Perseid meteor shower 2020: What time and how ...,en,4
...,...,...,...,...,...,...,...
108769,NATION,https://www.vanguardngr.com/2020/08/pdp-govern...,vanguardngr.com,2020-08-08 02:40:00,PDP governors’ forum urges security agencies t...,en,108769
108770,BUSINESS,https://www.patentlyapple.com/patently-apple/2...,patentlyapple.com,2020-08-08 01:27:12,"In Q2-20, Apple Dominated the Premium Smartpho...",en,108770
108771,HEALTH,https://www.belfastlive.co.uk/news/health/coro...,belfastlive.co.uk,2020-08-12 17:01:00,Coronavirus Northern Ireland: Full breakdown s...,en,108771
108772,ENTERTAINMENT,https://www.thenews.com.pk/latest/696364-paul-...,thenews.com.pk,2020-08-05 04:59:00,Paul McCartney details post-Beatles distress a...,en,108772


Para fines de este análisis, utilizaremos solo las primeras 1000 muestras del conjunto de datos "Newscatcher". Esta selección se hace con el objetivo de simplificar el proceso y permitir una exploración más rápida y eficiente de los datos. Limitar el análisis a un subconjunto más pequeño facilita la gestión y el procesamiento de los datos, sin comprometer la capacidad de obtener insights significativos.


In [4]:

pdf_subset = pdf.head(1000)

# InputExample en sentence-transformers

La clase `InputExample` en la biblioteca `sentence-transformers` se utiliza para encapsular un par o una tríada de oraciones que pueden ser utilizadas durante el entrenamiento o la evaluación de modelos de transformadores de oraciones.

In [5]:

def example_create_fn(doc1: pd.Series) -> InputExample:
    """
    Helper function that outputs a sentence_transformer guid, label, and text
    """
    return InputExample(texts=[doc1])

In [6]:
faiss_train_examples = pdf_subset.apply(
        lambda x: example_create_fn(x["title"]), axis=1
    ).tolist()

# faiss_train_examples

Cargamos el modelo de embedding

In [7]:
model = SentenceTransformer(
    "all-MiniLM-L6-v2", 
    # cache_folder="/path/to/cached/model" # en caso que se encuentre el modelo persistido
) 

In [11]:
# model.save("cache") # en caso que se necesite persistirlo

En esta etapa se realiza el embedding de los títulos y los mismos son llevados a un array de numpy con los valores de los 1000 embeddings

In [8]:
faiss_title_embedding = model.encode(pdf_subset.title.values.tolist())

In [9]:
faiss_title_embedding

array([[-0.11270548,  0.04076542,  0.02181416, ..., -0.01874594,
        -0.03136871,  0.0682483 ],
       [-0.02187161, -0.03349997,  0.07321803, ...,  0.03362321,
        -0.00563887, -0.00630978],
       [ 0.01608375,  0.00279447, -0.01504419, ..., -0.00706246,
         0.00905905, -0.0283505 ],
       ...,
       [ 0.01506922,  0.04583017, -0.061145  , ..., -0.07814185,
        -0.08025028,  0.0133782 ],
       [-0.07082237,  0.00643819,  0.0080932 , ..., -0.05520818,
        -0.03652041,  0.07594126],
       [-0.06321977,  0.04461521, -0.07385815, ...,  0.06559424,
         0.03276762,  0.09070996]], dtype=float32)

Cada embedding representa un vector multidimensional de 384 componentes para este modelo en particular, mientras que para modelos más grandes los espacios de dimensiones de embeddings pueden superar los miles

In [10]:
len(faiss_title_embedding), len(faiss_title_embedding[0])


(1000, 384)

Aquí seteamos el índice que vamos a utilizar para poder reconocer cada documento dentro de la base de datos, continúa siendo un DataFrame de pandas

In [11]:
pdf_to_index = pdf_subset.set_index(["id"], drop=False)

In [12]:
id_index = np.array(pdf_to_index.id.values).flatten().astype("int")

In [13]:
content_encoded_normalized = faiss_title_embedding.copy()

In [14]:
content_encoded_normalized

array([[-0.11270548,  0.04076542,  0.02181416, ..., -0.01874594,
        -0.03136871,  0.0682483 ],
       [-0.02187161, -0.03349997,  0.07321803, ...,  0.03362321,
        -0.00563887, -0.00630978],
       [ 0.01608375,  0.00279447, -0.01504419, ..., -0.00706246,
         0.00905905, -0.0283505 ],
       ...,
       [ 0.01506922,  0.04583017, -0.061145  , ..., -0.07814185,
        -0.08025028,  0.0133782 ],
       [-0.07082237,  0.00643819,  0.0080932 , ..., -0.05520818,
        -0.03652041,  0.07594126],
       [-0.06321977,  0.04461521, -0.07385815, ...,  0.06559424,
         0.03276762,  0.09070996]], dtype=float32)

In [15]:
faiss.normalize_L2(content_encoded_normalized)

# Uso de faiss.IndexIDMap y faiss.IndexFlatIP en FAISS

FAISS (Facebook AI Similarity Search) es una biblioteca diseñada para ayudar en la búsqueda de similitud y la agrupación de vectores grandes. En particular, `faiss.IndexIDMap` y `faiss.IndexFlatIP` son componentes clave para ciertas tareas.

## faiss.IndexIDMap

`faiss.IndexIDMap` se utiliza para mapear vectores a sus identificadores correspondientes (IDs). Esto es útil cuando se necesita mantener una referencia a los vectores originales en el conjunto de datos. En lugar de solo recuperar los vecinos más cercanos en términos de similitud, `faiss.IndexIDMap` permite que también se devuelvan los IDs asociados.

## faiss.IndexFlatIP

`faiss.IndexFlatIP` es un índice que utiliza el producto interno (`IP`) para calcular similitudes. Esto es particularmente útil cuando se desea medir la similitud coseno entre vectores, especialmente en tareas como la búsqueda de similitud de texto. El índice proporciona una búsqueda exhaustiva en un espacio vectorial, devolviendo los vecinos más cercanos en términos de similitud de producto interno.

En resumen, `faiss.IndexIDMap` ayuda a mantener la correspondencia entre vectores y sus IDs originales, mientras que `faiss.IndexFlatIP` facilita la búsqueda de similitud utilizando el producto interno. La combinación de ambos puede ser poderosa para aplicaciones donde se requiere la búsqueda eficiente de vecinos más cercanos junto con la correspondencia de IDs.


In [16]:
index_content = faiss.IndexIDMap(faiss.IndexFlatIP(len(faiss_title_embedding[0])))

In [17]:
index_content.add_with_ids(content_encoded_normalized, id_index)

In [18]:
def search_content(query, pdf_to_index, k=3):
    query_vector = model.encode([query])
    faiss.normalize_L2(query_vector)

    top_k = index_content.search(query_vector, k)
    ids = top_k[1][0].tolist()
    similarities = top_k[0][0].tolist()
    results = pdf_to_index.loc[ids]
    results["similarities"] = similarities
    return results

In [19]:
display(search_content("animal", pdf_to_index))

,topic,link,domain,published_date,title,lang,id,similarities
id,,,,,,,,
176,TECHNOLOGY,https://www.pushsquare.com/news/2020/08/random...,pushsquare.com,2020-08-03 16:30:00,Random: You Can Pick Up and Pet Cats in Assass...,en,176,0.391902
975,HEALTH,https://www.news-medical.net/news/20200813/Res...,news-medical.net,2020-08-13 05:18:00,Researchers explore social behavior of animals...,en,975,0.376784
99,TECHNOLOGY,https://www.gematsu.com/2020/08/ghostwire-toky...,gematsu.com,2020-08-07 16:43:13,Ghostwire: Tokyo confirms dog petting,en,99,0.344059


In [20]:
result = search_content("animal", pdf_to_index)

for id in result.id:
    print(result.loc[id, "title"])

Random: You Can Pick Up and Pet Cats in Assassin's Creed Valhalla
Researchers explore social behavior of animals toward emerging infectious diseases
Ghostwire: Tokyo confirms dog petting


In [21]:
result = search_content("money", pdf_to_index)

for id in result.id:
    print(result.loc[id, "title"])

Nintendo profit soars as people play more games staying home during the pandemic
Teenager Spends Almost $20,000 USD in Twitch Donations With Mother's Debit Card
The Fascinating Web Of Entropia Universe, The World’s Only ‘Cash-Based’ MMO


In [22]:
result = search_content("videogame", pdf_to_index)

for id in result.id:
    print(result.loc[id, "title"])

10 Brilliant Video Games That Surprised Everyone - Commenter Edition
Skater XL review: skateboarding video game is an overpriced Tony Hawk wannabe
A new ‘Call Of Duty’ alternate reality game has launched


In [23]:
result = search_content("bombs", pdf_to_index)

for id in result.id:
    print(result.loc[id, "title"])

NASA Releases In-Depth Map of Beirut Explosion Damage
NASA shows the impact of Beirut blast using satellite data
NASA Finds Ammonia-Linked 'Mushballs' and 'Shallow Lightning' on Jupiter
